# <div class="usecase-title">Mental Wellbeing</div>

<div class="usecase-authors"><b>Authored by: </b>Harley Ngo</div>

<div class="usecase-date"><b>Date: </b> March 2023</div>

<div class="usecase-duration"><b>Duration:</b> 90 mins</div>

<div class="usecase-level-skill"></div>
<div class="usecase-level"><b> Level: </b>Intermediate</div>
<div class="usecase-skill"><b> Pre-requisite Skills: </b>Python

<div class="usecase-subsection-blurb">
  <i>Dataset 1:</i> Indicators of wellbeing by year (Future Melbourne)
  <br>
  <a href="https://data.melbourne.vic.gov.au/explore/dataset/indicators-of-wellbeing-by-year-future-melbourne/table/" target="_blank">Dataset Link</a>
  <br>
</div>
<br>

<div class="usecase-subsection-blurb">
  <i>Dataset 2:</i> Indicators of quality of life and city services by year
  <br>
  <a href="https://data.melbourne.vic.gov.au/explore/dataset/indicators-of-quality-of-life-and-city-services-by-year/table/" target="_blank">Dataset Link</a>
  <br>
</div>
<br>

# <div class="usecase-section-header"><h4>Project Objective, Overview & Research</h4></div>

<div class="usecase-subtitle">
<i><b>User Story</b></i>
</div>

<div class="usecase-subsection-blurb"><i>"As a member of the city council, I want to review indicators of quality of life and city services by year, so I can prioritize funding and resources for initiatives that will enhance residents' living standards."</i></div>

<div class="usecase-subsection-blurb"><i>"As a resident of Future Melbourne, I want the city administration to use data-driven insights from indicators of wellbeing and quality of life to implement policies and services that address the needs of diverse communities and improve overall livability."</i></div>

<div class="usecase-subtitle">
<i><b>Objective</b></i>

<div class="usecase-subsection-blurb">'Mental Wellbeing' project contained two components, created over 3 weeks investigation period. The objective is to analyze the collected data, generate insights, and derive meaningful conclusions to guide future interventions.</div>

In [1]:
#Import core libraries
import requests
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from ipywidgets import interact, widgets
from IPython.display import display

from tqdm.auto import tqdm
from io import StringIO

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Download datasets
def download_dataset(api_key, dataset_id, base_url='https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'):
    format = 'csv'
    url = f'{base_url}{dataset_id}/exports/{format}'
    params = {
        'select': '*',
        'limit': -1,  # all records
        'lang': 'en',
        'timezone': 'UTC',
        'api_key': ''
    }

    with requests.get(url, params=params, stream=True) as response:
        if response.status_code == 200:
            total_size = int(response.headers.get('content-length', 0))
            chunk_size = 1024  # 1KB per chunk
            progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True, desc=f"Downloading {dataset_id}")

            content = bytearray()
            for chunk in response.iter_content(chunk_size=chunk_size):
                if chunk:  # filter out keep-alive new chunks
                    content.extend(chunk)
                    progress_bar.update(len(chunk))

            progress_bar.close()
            data = pd.read_csv(StringIO(content.decode('utf-8')), delimiter=';')
            return data
        else:
            print(f'Request failed with status code {response.status_code}')
            return None

# Dataset IDs
dataset_ids = [
    'indicators-of-wellbeing-by-year-future-melbourne',
    'indicators-of-quality-of-life-and-city-services-by-year'
]

# Initialize dictionary to hold the datasets
datasets = {}

# Download each dataset with a progress bar
for dataset_id in dataset_ids:
    datasets[dataset_id] = download_dataset(' ', dataset_id)
    if datasets[dataset_id] is not None:
        print(f"{dataset_id} downloaded successfully.")

indicators-of-wellbeing-by-year-future-melbourne downloaded successfully.


indicators-of-quality-of-life-and-city-services-by-year downloaded successfully.


# Part 1: Overall Analysis

## Dataset 1: Indicators of wellbeing by year (Future Melbourne)

In [3]:
wellbeing = datasets['indicators-of-wellbeing-by-year-future-melbourne']
wellbeing.head()

,indicator_label,indicator_description,unit,2012,2013,2014,2015,2016,population,source
0,Quality of life,Personal subjective wellbeing Index (average o...,%,75.6,76.7,76.8,76.9,76.5,Resident,Future Melbourne Social Survey
1,Quality of life (Male),Personal subjective wellbeing Index (average o...,%,75.6,76.7,77.4,76.6,75.9,Resident,Future Melbourne Social Survey
2,Quality of life (18-35 yrs),Personal subjective wellbeing Index (average o...,%,75.6,74.9,76.9,76.1,76.3,Resident,Future Melbourne Social Survey
3,Quality of life (36 yrs and over),Personal subjective wellbeing Index (average o...,%,75.5,78.3,76.8,77.6,76.6,Resident,Future Melbourne Social Survey
4,Personal relationships,How satisfied people feel with their personal ...,%,77.0,79.8,80.0,78.9,79.5,Resident,Future Melbourne Social Survey


In [4]:
def plot_indicator(indicator_label):
    indicator_data = wellbeing[wellbeing['indicator_label'] == indicator_label]
    
    # Extract years and corresponding values
    years = indicator_data.columns[3:-2].astype(int).values
    values = indicator_data.iloc[0, 3:-2].values.astype(float)
    
    # Fit ARIMA model
    model = ARIMA(values, order=(5,1,0)) 
    model_fit = model.fit()
    
    # Forecast for next years
    future_years = np.arange(years[-1] + 1, years[-1] + 6)
    future_values = model_fit.forecast(steps=5)  
    
    # Plot the line chart
    plt.figure(figsize=(10, 6))
    plt.plot(years, values, marker='o', label='Actual Data')
    plt.plot(future_years, future_values, linestyle='--', marker='o', color='r', label='Forecast')
    plt.title(f'Trend for {indicator_label}')
    plt.xlabel('Year')
    plt.ylabel('Percentage')
    plt.legend()
    plt.grid(True)
    plt.show()

indicator_labels = wellbeing['indicator_label'].unique().tolist()

dropdown = widgets.Dropdown(options=indicator_labels, description='Select Indicator Label:')

def update_plot(indicator_label):
    plot_indicator(indicator_label)

interactive_plot = interact(update_plot, indicator_label=dropdown)
display(interactive_plot)

interactive(children=(Dropdown(description='Select Indicator Label:', options=('Quality of life', 'Quality of …

<function __main__.update_plot(indicator_label)>

## Dataset 2: Indicators of quality of life and city services by year

In [5]:
life = datasets['indicators-of-quality-of-life-and-city-services-by-year']
life.head()

,number,indicator_theme,indicator_variable,indicator_descriptor,type,greater_melbourne_value,greater_melbourne_year,greater_melbourne_units,melbourne_municipality_value,melbourne_municipality_year,melbourne_municipality_units
0,5.2,Economy,Commercial and Industrial Property Values,Assessed value of commercial and industrial pr...,Core,NaN,NaN,%,47.5365,2015.0,NaN
1,5.4,Economy,Full-time Employment,Percentage of persons in fulltime employment,Supporting,50.6319,2014.0,%,58.5449,2014.0,NaN
2,19.4,Urban Planning,Jobs/Housing Ratio,Jobs/housing ratio,Supporting,1.2015,2015.0,NaN,7.6303,2012.0,NaN
3,5.6,Economy,Number of Businesses,Number of businesses per 100 000 population,Supporting,3584.4199,2014.0,/100 000,11500.1596,2014.0,NaN
4,5.7,Economy,Number of New Patents,Number of new patents per 100 000 population p...,Supporting,13.6251,2014.0,/100 000/yr,84.2832,2014.0,NaN


### Economy

In [6]:
life[life['indicator_theme'] == "Economy"].sort_values(by='number')

,number,indicator_theme,indicator_variable,indicator_descriptor,type,greater_melbourne_value,greater_melbourne_year,greater_melbourne_units,melbourne_municipality_value,melbourne_municipality_year,melbourne_municipality_units
61,5.1,Economy,Unemployment Rate,City's unemployment rate,Core,6.6497,2014.0,%,5.0346,2014.0,NaN
0,5.2,Economy,Commercial and Industrial Property Values,Assessed value of commercial and industrial pr...,Core,NaN,NaN,%,47.5365,2015.0,NaN
29,5.3,Economy,Population Living in Poverty,Percentage of city population living in poverty,Core,8.2000,2014.0,%,11.0223,2014.0,NaN
1,5.4,Economy,Full-time Employment,Percentage of persons in fulltime employment,Supporting,50.6319,2014.0,%,58.5449,2014.0,NaN
62,5.5,Economy,Youth Unemployment Rate,Youth unemployment rate,Supporting,11.0454,2011.0,%,7.8182,2011.0,NaN
3,5.6,Economy,Number of Businesses,Number of businesses per 100 000 population,Supporting,3584.4199,2014.0,/100 000,11500.1596,2014.0,NaN
4,5.7,Economy,Number of New Patents,Number of new patents per 100 000 population p...,Supporting,13.6251,2014.0,/100 000/yr,84.2832,2014.0,NaN


- The unemployment rate in the city was 6.6497% in 2014, with a higher rate of 11.0223% in the Melbourne municipality. This suggests a potential disparity in employment opportunities between the broader Greater Melbourne area and the specific municipality.
- Commercial and industrial property values experienced a significant increase of 47.5365% in 2015 in the Melbourne municipality. This substantial rise could indicate a robust economic environment or increased investment in commercial and industrial sectors within the municipality.
- The percentage of the population living in poverty was 8.2% in 2014 for Greater Melbourne, whereas it was higher at 11.0223% in the Melbourne municipality. This indicates a need for targeted poverty alleviation measures within the municipality.
- The percentage of persons in full-time employment was 50.6319% in 2014 for Greater Melbourne and increased to 58.5449% in the Melbourne municipality. This suggests a higher concentration of full-time employment opportunities within the municipality compared to the broader metropolitan area.
- The youth unemployment rate was notably higher at 11.0454% in 2011 compared to the overall unemployment rate, indicating potential challenges in youth employment within the region.
- The number of businesses per 100,000 population was substantially higher in the Melbourne municipality compared to Greater Melbourne, indicating a concentration of business activity within the municipality.
- The number of new patents per 100,000 population per year was relatively low in both Greater Melbourne and the Melbourne municipality, suggesting potential areas for fostering innovation and research within the region.

<b><i>Conclusion: To address disparities and foster growth within the Melbourne municipality, the city council could focus on targeted employment initiatives, youth empowerment, poverty alleviation, innovation support, infrastructure investment, small business promotion, and data-driven decision-making. These efforts aim to enhance economic opportunities, reduce poverty, stimulate innovation, and ensure sustainable development across the municipality.</i></b>

### Education

In [7]:
life[life['indicator_theme'] == "Education"].sort_values(by='number')

,number,indicator_theme,indicator_variable,indicator_descriptor,type,greater_melbourne_value,greater_melbourne_year,greater_melbourne_units,melbourne_municipality_value,melbourne_municipality_year,melbourne_municipality_units
63,6.1,Education,Female Student Enrolment,Percentage of female school-aged population en...,Core,100.0156,2013.0,%,NaN,NaN,NaN
64,6.2,Education,Primary Education Completion,Percentage of students completing primary educ...,Core,102.0663,2013.0,%,NaN,NaN,NaN
5,6.3,Education,Secondary Education Completion,Percentage of students completing secondary ed...,Core,91.8848,2013.0,%,70.0492,2013.0,NaN
65,6.4,Education,Student/Teacher Ratio,Primary education student/teacher ratio,Core,15.1977,2013.0,NaN,14.7139,2013.0,NaN
66,6.5,Education,Male Student Enrolment,Percentage of male school-aged population enro...,Supporting,99.8322,2013.0,%,NaN,NaN,NaN
6,6.6,Education,Total Student Enrolment,Percentage of school-aged population enrolled ...,Supporting,99.9218,2013.0,%,NaN,NaN,NaN
7,6.7,Education,Higher Education Degrees,Number of higher education degrees per 100 000...,Supporting,26388.0731,2014.0,/100 000,46630.7167,2014.0,NaN


- Female student enrolment exceeded 100% in 2013 in Greater Melbourne, indicating potential discrepancies or data anomalies.
- Primary education completion rate was 102.07% in 2013, suggesting possible overestimation or exceptional circumstances.
- Secondary education completion rate was 91.88% in Greater Melbourne and 70.05% in the Melbourne municipality in 2013, showing differences in completion rates.
- The student/teacher ratio was approximately 15 in primary education and slightly lower in the Melbourne municipality in 2013.
- Male student enrolment was around 99.83% in 2013, indicating near parity with female enrolment.
- Total student enrolment was approximately 99.92% in 2013, suggesting high enrolment rates.
- Higher education degrees were more prevalent in the Melbourne municipality, with 46,630.72 per 100,000 population compared to 26,388.07 in Greater Melbourne in 2014.

<b><i>Conclusion: The council should implement targeted support programs to assist students, particularly within the Melbourne municipality, in completing secondary education. This effort should be complemented by an evaluation of teacher allocations to ensure balanced student-teacher ratios, especially in areas with higher needs. Additionally, promoting gender equity in education and encouraging participation in underrepresented fields is essential, alongside facilitating partnerships and providing support for higher education pursuits. Continuous monitoring of educational outcomes will enable the council to identify trends and areas for improvement, guiding future interventions and resource allocation effectively.</i></b>

### Energy

In [8]:
life[life['indicator_theme'] == "Energy"].sort_values(by='number')

,number,indicator_theme,indicator_variable,indicator_descriptor,type,greater_melbourne_value,greater_melbourne_year,greater_melbourne_units,melbourne_municipality_value,melbourne_municipality_year,melbourne_municipality_units
8,7.1,Energy,Residential Electrical Use,Total residential electrical energy use per ca...,Core,NaN,NaN,kWh/yr/capita,1730.8241,2015.0,NaN
9,7.2,Energy,Authorized Electrical Service,Percentage of city population with authorized ...,Core,100.0000,2013.0,%,100.0000,2013.0,NaN
30,7.3,Energy,Public Building Electricity Use,Energy (electricity) consumption of public bui...,Core,NaN,NaN,kWh/m²,360.7616,2014.0,NaN
10,7.4,Energy,Electricity from Renewable Sources,Percentage of total energy derived from renewa...,Core,NaN,NaN,%,12.3500,2014.0,NaN
11,7.5,Energy,Total Electrical Use,Total electrical energy use per capita (kWh/year),Supporting,NaN,NaN,kWh/yr/capita,27083.7691,2014.0,NaN
31,7.6,Energy,Electrical Interruptions,Average number of electrical interruptions per...,Supporting,1.3069,2010.0,/customer/yr,0.2810,2010.0,NaN
12,7.7,Energy,Electrical Interruption Length,Average length of electrical interruptions (in...,Supporting,1.6329,2010.0,hours,1.8827,2010.0,NaN


- The average number of electrical interruptions per customer per year was 1.31 in Greater Melbourne in 2010 and 0.28 in the Melbourne municipality, indicating a difference in service reliability.
- The average length of electrical interruptions was 1.63 hours in Greater Melbourne in 2010 and 1.88 hours in the Melbourne municipality, suggesting slight variations in outage durations.

<b><i>Conclusion: To address discrepancies in service reliability within Greater Melbourne, the city council should prioritize infrastructure improvements and maintenance initiatives. This could involve targeted investments in the Melbourne municipality to reduce the average number of electrical interruptions per customer per year, potentially through upgrades to aging infrastructure or enhanced monitoring systems. Similarly, efforts to minimize outage durations should be emphasized, with a focus on areas experiencing longer interruptions. Collaborating with utility providers and implementing proactive measures to address underlying issues contributing to service disruptions will be crucial in ensuring consistent and reliable electrical supply across the municipality.</i></b>

### Environment

In [9]:
life[life['indicator_theme'] == "Environment"].sort_values(by='number')

,number,indicator_theme,indicator_variable,indicator_descriptor,type,greater_melbourne_value,greater_melbourne_year,greater_melbourne_units,melbourne_municipality_value,melbourne_municipality_year,melbourne_municipality_units
67,8.1,Environment,PM2.5 Concentration,Fine Particulate Matter (PM2.5) concentration,Core,7.4000,2014.0,?g/m³,7.4000,2014.0,NaN
68,8.2,Environment,PM10 Concentration,Particulate Matter (PM10) concentration,Core,17.4000,2014.0,?g/m³,17.4000,2014.0,NaN
69,8.3,Environment,GHG Emissions,Greenhouse gas emissions measured in tonnes pe...,Core,NaN,NaN,t/capita,42.7262,2014.0,NaN
32,8.4,Environment,NO2 Concentration,NO? (nitrogen dioxide) concentration,Supporting,15.8693,2014.0,?g/m³,15.8693,2014.0,NaN
70,8.5,Environment,SO2 Concentration,SO? (sulphur dioxide) concentration,Supporting,2.1014,2014.0,?g/m³,2.1014,2014.0,NaN
71,8.6,Environment,O3 Concentration,O? (ozone) concentration,Supporting,34.3945,2014.0,?g/m³,34.3945,2014.0,NaN
72,8.7,Environment,Noise Pollution,Noise pollution,Supporting,NaN,NaN,%,NaN,NaN,NaN
33,8.8,Environment,Change in Native Species,Percentage change in number of native species,Supporting,NaN,NaN,%,NaN,NaN,NaN


- Air quality indicators like PM2.5, PM10, NO2, SO2, and O3 concentrations were consistent between Greater Melbourne and its municipalities in 2014, suggesting uniform environmental conditions.

<b><i>Conclusion: The city council should prioritize ongoing monitoring of air quality parameters and collaborate with relevant stakeholders to implement measures aimed at preserving and improving air quality standards. This could involve initiatives such as promoting sustainable transportation, reducing industrial emissions, and enhancing green infrastructure to mitigate potential environmental risks and ensure the health and well-being of residents across Greater Melbourne.</i></b>

### Finance

In [10]:
life[life['indicator_theme'] == "Finance"].sort_values(by='number')

,number,indicator_theme,indicator_variable,indicator_descriptor,type,greater_melbourne_value,greater_melbourne_year,greater_melbourne_units,melbourne_municipality_value,melbourne_municipality_year,melbourne_municipality_units
73,9.1,Finance,Debt Service Ratio,Debt service ratio (debt service expenditure a...,Core,1.0252,2015.0,%,0.0000,2015.0,NaN
74,9.2,Finance,Capital Spending,Capital spending as a percentage of total expe...,Supporting,20.2772,2015.0,%,17.9328,2015.0,NaN
34,9.3,Finance,Own-Source Revenues,Own-source revenue as a percentage of total re...,Supporting,75.1170,2015.0,%,86.5048,2015.0,NaN
75,9.4,Finance,Taxes Collected,Tax collected as percentage of tax billed,Supporting,99.6555,2015.0,%,100.9082,2015.0,NaN


- The debt service ratio differs notably between Greater Melbourne and the Melbourne municipality in 2015, indicating potentially divergent financial management approaches or debt burdens across regions.
- While both areas allocated a significant portion of expenditure to capital spending in 2015, the Melbourne municipality's slightly lower percentage suggests a nuanced difference in investment priorities or fiscal strategies.
- Greater Melbourne relied less on own-source revenue compared to the Melbourne municipality in 2015, indicating potential differences in revenue diversification strategies or economic structures between the two areas.
- Both areas demonstrated high tax collection efficiency, but the Melbourne municipality's slightly higher percentage suggests a slightly more effective tax administration system or compliance rate.

<b><i>Conclusion: The differences in the debt service ratio, capital spending allocation, own-source revenue reliance, and tax collection efficiency between Greater Melbourne and the Melbourne municipality in 2015 suggest varying financial management approaches and priorities. To address these disparities and promote fiscal sustainability, the city council should analyze financial data and develop tailored strategies to enhance financial management practices, optimize resource allocation, and improve revenue generation and tax administration effectiveness across the region.</i></b>

### Fire and Emergency Response

In [11]:
life[life['indicator_theme'] == "Fire and Emergency Response"].sort_values(by='number')

,number,indicator_theme,indicator_variable,indicator_descriptor,type,greater_melbourne_value,greater_melbourne_year,greater_melbourne_units,melbourne_municipality_value,melbourne_municipality_year,melbourne_municipality_units
35,10.1,Fire and Emergency Response,Number of Firefighters,Number of firefighters per 100 000 population,Core,NaN,NaN,/100 000,278.2165,2014.0,NaN
13,10.2,Fire and Emergency Response,Number of Fire-Related Deaths,Number of fire related deaths per 100 000 popu...,Core,0.2116,2012.0,/100 000/yr,0.0000,2012.0,NaN
36,10.3,Fire and Emergency Response,Natural Disaster-Related Deaths,Number of natural disaster related deaths per ...,Core,0.0000,2012.0,/100 000/yr,0.0000,2012.0,NaN
14,10.4,Fire and Emergency Response,Volunteer Firefighters,Number of volunteer and part-time firefighters...,Supporting,NaN,NaN,/100 000,0.0000,2013.0,NaN
76,10.5,Fire and Emergency Response,Emergency Response Time,Response time for emergency response services ...,Supporting,NaN,NaN,min,NaN,NaN,NaN
77,10.6,Fire and Emergency Response,Fire Response Time,Response time for fire department from initial...,Supporting,NaN,NaN,min,NaN,NaN,NaN


- The Melbourne municipality reported a lower rate of fire-related deaths compared to Greater Melbourne in 2012. This indicates potentially more effective fire safety measures or emergency response systems in the municipality.

### Governance

In [12]:
life[life['indicator_theme'] == "Governance"].sort_values(by='number')

,number,indicator_theme,indicator_variable,indicator_descriptor,type,greater_melbourne_value,greater_melbourne_year,greater_melbourne_units,melbourne_municipality_value,melbourne_municipality_year,melbourne_municipality_units
78,11.1,Governance,Voter Participation Rate,Voter participation in last municipal election...,Core,69.0133,2012.0,%,60.0098,2012.0,NaN
15,11.2,Governance,Women Elected to Office,Women as a percentage of total elected to city...,Core,36.8771,2012.0,%,36.3636,2015.0,NaN
16,11.3,Governance,Women Employed in City Government,Percentage of women employed in the city gover...,Supporting,NaN,NaN,%,38.8065,2013.0,NaN
37,11.4,Governance,Number of Corruption Convictions,Number of convictions for corruption and / or ...,Supporting,NaN,NaN,/100 000,NaN,NaN,NaN
79,11.5,Governance,Citizen Representation,Citizens' representation: number of local offi...,Supporting,7.0783,2012.0,/100 000,9.0011,2014.0,NaN
38,11.6,Governance,Number of Registered Voters,Number of registered voters as a percentage of...,Supporting,87.6410,2012.0,%,129.6933,2012.0,NaN


- Voter participation rates were higher in Greater Melbourne compared to the Melbourne municipality in 2012, indicating varying levels of civic engagement across regions. This suggests a need for targeted initiatives to encourage voter participation, especially in areas with lower turnout.
- Both Greater Melbourne and the Melbourne municipality demonstrated relatively similar levels of gender representation in elected offices in 2012. However, efforts to improve gender inclusivity in city government employment were evident in the Melbourne municipality in 2013. This highlights ongoing endeavors to enhance gender diversity in municipal governance roles.
- Data on corruption convictions were not available for Greater Melbourne, suggesting a potential gap in transparency measures or reporting standards. Enhancing transparency and anti-corruption efforts could bolster public trust and accountability in governance.
- The number of local officials per 100,000 citizens varied between Greater Melbourne and the Melbourne municipality in 2012, indicating differences in citizen representation structures. Understanding these variations can inform discussions on optimizing governance structures to better serve the population's needs.
- The percentage of registered voters was notably higher in the Melbourne municipality compared to Greater Melbourne in 2012. This discrepancy raises questions about voter registration practices and data accuracy, emphasizing the importance of ensuring inclusive and accurate voter registration processes.

### Health

In [13]:
life[life['indicator_theme'] == "Health"].sort_values(by='number')

,number,indicator_theme,indicator_variable,indicator_descriptor,type,greater_melbourne_value,greater_melbourne_year,greater_melbourne_units,melbourne_municipality_value,melbourne_municipality_year,melbourne_municipality_units
39,12.1,Health,Life Expectancy,Average life expectancy,Core,83.5000,2013.0,years,83.5000,2013.0,NaN
80,12.2,Health,In-Patient Hospital Beds,Number of in-patient hospital beds per 100 000...,Core,349.7489,2014.0,/100 000,2114.4397,2012.0,NaN
17,12.3,Health,Number of Physicians,Number of physicians per 100 000 population,Core,369.9119,2014.0,/100 000,2514.5041,2014.0,NaN
40,12.4,Health,Under Age Five Mortality Rate,Under age five mortality per 1 000 live births,Core,3.3000,2013.0,/1000 live births,5.4000,2013.0,NaN
41,12.5,Health,Number of Nurses/Midwives,Number of nursing and midwifery personnel per ...,Supporting,979.5966,2014.0,/100 000,5024.5894,2014.0,NaN
18,12.6,Health,Number of Mental Health Practitioners,Number of mental health practitioners per 100 ...,Supporting,188.5469,2014.0,/100 000,1090.9359,2014.0,NaN
81,12.7,Health,Suicide Rate,Suicide rate per 100 000 population,Supporting,8.3951,2012.0,/100 000,8.5387,2012.0,NaN


- The average life expectancy was consistent at 83.5 years for both Greater Melbourne and the Melbourne municipality in 2013, indicating comparable health outcomes across regions.
- The number of in-patient hospital beds per 100,000 population was significantly higher in the Melbourne municipality (2114.44 beds) compared to Greater Melbourne (349.75 beds) in 2014. This suggests potentially greater healthcare infrastructure and capacity in the municipality.
- Greater Melbourne had 369.91 physicians per 100,000 population in 2014, while the Melbourne municipality had a notably higher number at 2514.50 physicians. This indicates differences in access to healthcare professionals between the two regions.
- The under-five mortality rate was lower in Greater Melbourne (3.3 per 1000 live births) compared to the Melbourne municipality (5.4 per 1000 live births) in 2013, reflecting differences in child healthcare and public health interventions.
- The number of nursing and midwifery personnel per 100,000 population was higher in the Melbourne municipality (5024.59 personnel) compared to Greater Melbourne (979.60 personnel) in 2014, indicating potentially greater healthcare staffing levels in the municipality.
- The number of mental health practitioners per 100,000 population was higher in the Melbourne municipality (1090.94 practitioners) compared to Greater Melbourne (188.55 practitioners) in 2014, suggesting potentially greater access to mental health services in the municipality.
- The suicide rate was comparable between Greater Melbourne (8.40 per 100,000 population) and the Melbourne municipality (8.54 per 100,000 population) in 2012, indicating consistent mental health challenges across regions.

<b><i>Conclusion: To address healthcare disparities across Greater Melbourne, the city council should invest in expanding healthcare infrastructure in areas with lower bed availability, while implementing strategies to attract healthcare professionals to underserved regions. Targeted initiatives should aim to improve child healthcare outcomes and mental health support, leveraging existing resources and promoting awareness. Continuous monitoring of key health indicators is essential to identify trends and allocate resources effectively, ensuring equitable access to quality healthcare services for all residents.</i></b>

### Recreation

In [14]:
life[life['indicator_theme'] == "Recreation"].sort_values(by='number')

,number,indicator_theme,indicator_variable,indicator_descriptor,type,greater_melbourne_value,greater_melbourne_year,greater_melbourne_units,melbourne_municipality_value,melbourne_municipality_year,melbourne_municipality_units
42,13.1,Recreation,Indoor Recreation Space,Square metres of public indoor recreation spac...,Supporting,NaN,NaN,m²/capita,8.5415,2012.0,NaN
19,13.2,Recreation,Outdoor Recreation Space,Square metres of public outdoor recreation spa...,Supporting,NaN,NaN,m²/capita,14.6141,2012.0,NaN


### Safety

In [15]:
life[life['indicator_theme'] == "Safety"].sort_values(by='number')

,number,indicator_theme,indicator_variable,indicator_descriptor,type,greater_melbourne_value,greater_melbourne_year,greater_melbourne_units,melbourne_municipality_value,melbourne_municipality_year,melbourne_municipality_units
82,14.1,Safety,Number of Police Officers,Number of police officers per 100 000 population,Core,132.5499,2014.0,/100 000/yr,253.8398,2014.0,NaN
20,14.2,Safety,Number of Homicides,Number of homicides per 100 000 population,Core,2.1395,2014.0,/100 000/yr,8.1828,2014.0,NaN
21,14.3,Safety,Number of Property Crimes,Crimes against property per 100 000 population,Supporting,4253.2894,2014.0,/100 000/yr,10721.9717,2014.0,NaN
43,14.4,Safety,Police Response Time,Response time for police department from initi...,Supporting,NaN,NaN,min,NaN,NaN,NaN
44,14.5,Safety,Number of Violent Crimes,Violent crime rate per 100 000 population,Supporting,1040.1259,2014.0,/100 000/yr,2849.2640,2014.0,NaN


- The number of police officers per 100,000 population was higher in the Melbourne municipality (253.84 officers) compared to Greater Melbourne (132.55 officers) in 2014. This indicates potentially greater law enforcement resources and coverage in the municipality.
- The homicide rate per 100,000 population was higher in the Melbourne municipality (8.18 homicides) compared to Greater Melbourne (2.14 homicides) in 2014. This suggests potentially differing levels of crime and safety concerns between the two regions.
- The rate of crimes against property per 100,000 population was higher in the Melbourne municipality (10721.97 crimes) compared to Greater Melbourne (4253.29 crimes) in 2014. This indicates potentially higher incidences of property-related offenses in the municipality.
- The rate of violent crimes per 100,000 population was higher in the Melbourne municipality (2849.26 crimes) compared to Greater Melbourne (1040.13 crimes) in 2014. This suggests potentially greater challenges with violent crime prevention and law enforcement in the municipality.

<b><i>Conclusion: To address these disparities and improve public safety, the city council should consider targeted strategies such as increasing police presence, enhancing community policing efforts, and implementing crime prevention initiatives tailored to the specific needs of each area.</i></b>

### Shelter

In [16]:
life[life['indicator_theme'] == "Shelter"].sort_values(by='number')

,number,indicator_theme,indicator_variable,indicator_descriptor,type,greater_melbourne_value,greater_melbourne_year,greater_melbourne_units,melbourne_municipality_value,melbourne_municipality_year,melbourne_municipality_units
45,15.1,Shelter,Population Living in Slums,Percentage of city population living in slums,Core,0.4705,2011.0,%,0.6184,2011.0,NaN
46,15.2,Shelter,Number of Homeless,Number of homeless per 100 000 population,Supporting,156.8800,2011.0,/100 000,395.0500,2011.0,NaN
83,15.3,Shelter,Unregistered Households,Percentage of households that exist without re...,Supporting,0.0000,2011.0,%,0.0000,2014.0,NaN


- In 2011, a small percentage of the population lived in slums in both Greater Melbourne (0.47%) and the Melbourne municipality (0.62%). This suggests relatively low incidences of slum living, indicating a generally favorable housing situation.
- In 2011, there were 156.88 homeless individuals per 100,000 population in Greater Melbourne and 395.05 homeless individuals per 100,000 population in the Melbourne municipality. This indicates a higher prevalence of homelessness in the municipality compared to the broader Greater Melbourne area.

<b><i>Conclusion: To mitigate homelessness disparities between the Melbourne municipality and Greater Melbourne, the city council should focus on expanding affordable housing programs, increasing funding for supportive services, and implementing homelessness prevention strategies. Investing in mental health, substance abuse treatment, and employment programs is crucial to address underlying factors contributing to homelessness. Collaboration with housing agencies and community stakeholders is essential to develop and implement effective solutions.</i></b>

### Solid Waste

In [17]:
life[life['indicator_theme'] == "Solid Waste"].sort_values(by='number')

,number,indicator_theme,indicator_variable,indicator_descriptor,type,greater_melbourne_value,greater_melbourne_year,greater_melbourne_units,melbourne_municipality_value,melbourne_municipality_year,melbourne_municipality_units
22,16.1,Solid Waste,Hazardous Waste Recycled,Percentage of city's hazardous waste that is r...,Supporting,NaN,NaN,%,NaN,NaN,NaN
84,16.1,Solid Waste,Residential Solid Waste Collection,Percentage of city population with regular sol...,Core,100.0000,2014.0,%,100.0000,2015.0,NaN
85,16.2,Solid Waste,Solid Waste Collected,Total collected municipal solid waste per capita,Core,0.3921,2013.0,t/capita,0.3384,2013.0,NaN
86,16.3,Solid Waste,Solid Waste Recycled,Percentage of the city's solid waste that is r...,Core,43.6805,2013.0,%,20.9427,2013.0,NaN
47,16.4,Solid Waste,Disposal by Landfill,Percentage of the city's solid waste that is d...,Supporting,56.3195,2013.0,%,79.0573,2013.0,NaN
48,16.5,Solid Waste,Disposal by Incinerator,Percentage of the city's solid waste that is d...,Supporting,0.0000,2013.0,%,0.0000,2013.0,NaN
49,16.6,Solid Waste,Disposal by Burning Openly,Percentage of the city's solid waste that is b...,Supporting,0.0000,2013.0,%,0.0000,2013.0,NaN
50,16.7,Solid Waste,Disposal by Open Dump,Percentage of the city's solid waste that is d...,Supporting,0.0000,2013.0,%,0.0000,2013.0,NaN
87,16.8,Solid Waste,Disposal by Other,Percentage of the city's solid waste that is d...,Supporting,0.0000,2013.0,%,0.0000,2013.0,NaN
88,16.9,Solid Waste,Hazardous Waste Generated,Hazardous waste generation per capita,Supporting,NaN,NaN,t/capita,NaN,NaN,NaN


- Both Greater Melbourne and the Melbourne municipality had 100% of their respective populations covered by regular solid waste collection services in 2014 and 2015. This indicates comprehensive waste management infrastructure and services in place to handle residential waste.
- Both Greater Melbourne and the Melbourne municipality demonstrated relatively similar levels of gender representation in elected offices in 2012. However, efforts to improve gender inclusivity in city government employment were evident in the Melbourne municipality in 2013. This highlights ongoing endeavors to enhance gender diversity in municipal governance roles.
- The percentage of registered voters was notably higher in the Melbourne municipality compared to Greater Melbourne in 2012. This discrepancy raises questions about voter registration practices and data accuracy, emphasizing the importance of ensuring inclusive and accurate voter registration processes.

### Telecommunication and Innovation

In [18]:
life[life['indicator_theme'] == "Telecommunication and Innovation"].sort_values(by='number')

,number,indicator_theme,indicator_variable,indicator_descriptor,type,greater_melbourne_value,greater_melbourne_year,greater_melbourne_units,melbourne_municipality_value,melbourne_municipality_year,melbourne_municipality_units
89,17.1,Telecommunication and Innovation,Internet Connections,Number of internet connections per 100 000 pop...,Core,53631.9513,2014.0,/100 000,53631.9513,2014.0,NaN
90,17.2,Telecommunication and Innovation,Cell Phone Connections,Number of cell phone connections per 100 000 p...,Core,84935.2122,2013.0,/100 000,84935.2122,2013.0,NaN
51,17.3,Telecommunication and Innovation,Landline Connections,Number of landline phone connections per 100 0...,Supporting,NaN,NaN,/100 000,NaN,NaN,NaN


- Both Greater Melbourne and the Melbourne municipality had a high number of internet connections per 100,000 population in 2014, with approximately 53,631.95 connections. This indicates widespread access to the internet, essential for communication, information access, and economic participation.
- Similarly, both Greater Melbourne and the Melbourne municipality had a high number of cell phone connections per 100,000 population in 2013, with approximately 84,935.21 connections. This suggests extensive mobile network coverage and high mobile phone adoption rates, facilitating communication and connectivity.

### Transportation

In [19]:
life[life['indicator_theme'] == "Transportation"].sort_values(by='number')

,number,indicator_theme,indicator_variable,indicator_descriptor,type,greater_melbourne_value,greater_melbourne_year,greater_melbourne_units,melbourne_municipality_value,melbourne_municipality_year,melbourne_municipality_units
91,18.1,Transportation,High-Capacity Public Transit,Kilometres of high capacity public transport s...,Core,28.4601,2015.0,km/100 000,102.8714,2015.0,NaN
92,18.2,Transportation,Light Passenger Public Transit,Kilometres of light passenger public transport...,Core,180.3500,2015.0,km/100 000,293.5700,2015.0,NaN
93,18.3,Transportation,Public Transit Trips,Annual number of public transport trips per ca...,Core,81.7152,2014.0,capita/yr,1009.1830,2014.0,NaN
52,18.4,Transportation,Number of Automobiles,Number of personal automobiles per capita,Core,0.5967,2013.0,/capita,0.7167,2013.0,NaN
23,18.5,Transportation,Mode Share,Percentage of commuters using a travel mode to...,Supporting,29.1323,2011.0,%,65.8252,2011.0,NaN
94,18.6,Transportation,Number of 2-Wheeled Vehicles,Number of two-wheel motorized vehicles per capita,Supporting,0.0251,2013.0,/capita,0.0249,2013.0,NaN
95,18.7,Transportation,Bicycle Paths,Kilometres of bicycle paths and lanes per 100 ...,Supporting,98.4824,2014.0,km/100 000,147.4204,2014.0,NaN
53,18.8,Transportation,Transportation Fatalities,Transportation fatalities per 100 000 population,Supporting,2.6239,2013.0,/100 000/yr,1.7191,2013.0,NaN
96,18.9,Transportation,Commercial Air Connectivity,Commercial air connectivity,Supporting,114368.0000,2014.0,/yr,114368.0000,2014.0,NaN


- In 2015, Greater Melbourne had 28.46 kilometers of high-capacity public transport services per 100,000 population, while the Melbourne municipality had a notably higher coverage of 102.87 kilometers. This suggests a more extensive high-capacity public transit network in the municipality, potentially providing better access to mass transit options for residents.
- In 2015, Greater Melbourne had 180.35 kilometers of light passenger public transport services per 100,000 population, while the Melbourne municipality had a higher coverage of 293.57 kilometers. This indicates a more comprehensive coverage of light passenger public transit options in the municipality.
- The annual number of public transport trips per capita was 81.72 in Greater Melbourne and substantially higher at 1009.18 in the Melbourne municipality in 2014. This reflects a higher usage of public transit in the municipality, possibly due to better service coverage and accessibility.
- Both Greater Melbourne and the Melbourne municipality had a relatively low number of personal automobiles per capita in 2013, with 0.5967 and 0.7167 automobiles per capita, respectively. This suggests a relatively low dependence on personal vehicles for transportation in both areas.
- In 2011, a higher percentage of commuters in the Melbourne municipality (65.83%) used a travel mode other than private automobiles, compared to Greater Melbourne (29.13%). This indicates a higher adoption of alternative transportation modes such as public transit, walking, or cycling in the municipality.
- The number of two-wheeled motorized vehicles per capita was relatively low in both Greater Melbourne and the Melbourne municipality in 2013, suggesting a limited prevalence of motorized two-wheeled vehicles compared to other forms of transportation.
- In 2014, the Melbourne municipality had a higher coverage of bicycle paths and lanes per 100,000 population (147.42 kilometers) compared to Greater Melbourne (98.48 kilometers). This suggests potentially better infrastructure for cycling in the municipality, promoting active transportation options.
- The transportation fatalities per 100,000 population were slightly higher in Greater Melbourne (2.62) compared to the Melbourne municipality (1.72) in 2013, indicating potential differences in road safety measures or infrastructure between the two areas.
- Both Greater Melbourne and the Melbourne municipality had the same level of commercial air connectivity, with 114,368 connections in 2014. This suggests similar access to air travel options for residents and businesses across the region.

<b><i>Conclusion: To improve transportation accessibility and promote sustainable mobility, the city council should focus on expanding public transit infrastructure, particularly in Greater Melbourne where coverage is comparatively lower. Investments in high-capacity and light passenger public transport services can enhance connectivity and increase ridership. Additionally, efforts to encourage alternative transportation modes, such as walking, cycling, and public transit, should be prioritized, leveraging existing low dependence on personal automobiles. Enhancing road safety measures and promoting active transportation options, such as cycling infrastructure, can further improve mobility and reduce transportation fatalities. Collaboration with relevant stakeholders is essential to develop and implement effective transportation policies and initiatives that cater to the diverse needs of residents across the Melbourne municipality and Greater Melbourne area.</i></b>

### Urban Planning

In [20]:
life[life['indicator_theme'] == "Urban Planning"].sort_values(by='number')

,number,indicator_theme,indicator_variable,indicator_descriptor,type,greater_melbourne_value,greater_melbourne_year,greater_melbourne_units,melbourne_municipality_value,melbourne_municipality_year,melbourne_municipality_units
54,19.1,Urban Planning,Green Area,Green area (hectares) per 100 000 population,Core,NaN,NaN,ha/100 000,451.9344,2015.0,NaN
55,19.2,Urban Planning,Trees Planted,Annual number of trees planted per 100 000 pop...,Supporting,NaN,NaN,/100 000/yr,2640.4785,2015.0,NaN
97,19.3,Urban Planning,Informal Settlements,Areal size of informal settlements as a percen...,Supporting,0.0000,2014.0,%,0.0000,2015.0,NaN
2,19.4,Urban Planning,Jobs/Housing Ratio,Jobs/housing ratio,Supporting,1.2015,2015.0,NaN,7.6303,2012.0,NaN


- In 2015, the Melbourne municipality planted a significantly higher number of trees per 100,000 population (2,640.48) compared to data availability for Greater Melbourne. This indicates a proactive approach to urban greening and tree canopy expansion within the municipality, which can contribute to improved air quality, biodiversity, and urban aesthetics.
- In 2015, the jobs/housing ratio was 1.2015 in Greater Melbourne and notably higher at 7.6303 in the Melbourne municipality in 2012. A higher ratio indicates a greater concentration of employment opportunities relative to housing units. This disparity may have implications for commuting patterns, urban sprawl, and access to employment opportunities within the municipality.

### Wastewater

In [21]:
life[life['indicator_theme'] == "Wastewater"].sort_values(by='number')

,number,indicator_theme,indicator_variable,indicator_descriptor,type,greater_melbourne_value,greater_melbourne_year,greater_melbourne_units,melbourne_municipality_value,melbourne_municipality_year,melbourne_municipality_units
56,20.1,Wastewater,Wastewater Collection,Percentage of city population served by wastew...,Core,100.0,2015.0,%,100.0,2015.0,NaN
57,20.2,Wastewater,No Wastewater Treatment,Percentage of the city's wastewater that has r...,Core,0.0,2014.0,%,0.0,2015.0,NaN
24,20.3,Wastewater,Primary Wastewater Treatment,Percentage of the city's wastewater receiving ...,Core,100.0,2014.0,%,100.0,2015.0,NaN
25,20.4,Wastewater,Secondary Wastewater Treatment,Percentage of the city's wastewater receiving ...,Core,100.0,2014.0,%,100.0,2015.0,NaN
58,20.5,Wastewater,Tertiary Wastewater Treatment,Percentage of the city's wastewater receiving ...,Core,NaN,NaN,%,100.0,2015.0,NaN


- Both Greater Melbourne and the Melbourne municipality achieved 100% coverage of wastewater collection in 2015. This indicates comprehensive infrastructure for collecting wastewater from households and businesses, contributing to public health and environmental protection by preventing contamination of water sources.
- Both areas reported 0% of wastewater left untreated in 2014 and 2015. This signifies that all wastewater generated within Greater Melbourne and the Melbourne municipality underwent some form of treatment before discharge, aligning with environmental regulations and standards.
- In 2014, 100% of the city's wastewater received primary treatment, indicating the removal of solid materials and debris. This initial treatment step is essential for reducing the organic load and improving the quality of wastewater before further processing.
- Similarly, in 2014, 100% of the city's wastewater underwent secondary treatment, which involves biological processes to further reduce pollutants and organic matter. This step is crucial for achieving higher water quality standards and minimizing environmental impacts.

### Water and Sanitation

In [22]:
life[life['indicator_theme'] == "Water and Sanitation"].sort_values(by='number')

,number,indicator_theme,indicator_variable,indicator_descriptor,type,greater_melbourne_value,greater_melbourne_year,greater_melbourne_units,melbourne_municipality_value,melbourne_municipality_year,melbourne_municipality_units
26,21.1,Water and Sanitation,Potable Water Service,Percentage of city population with potable wat...,Core,100.0000,2014.0,%,100.0000,2014.0,NaN
59,21.2,Water and Sanitation,Improved Water Access,Percentage of city population with sustainable...,Core,100.0000,2014.0,%,100.0000,2014.0,NaN
27,21.3,Water and Sanitation,Improved Sanitation Access,Percentage of population with access to improv...,Core,100.0000,2014.0,%,100.0000,2015.0,NaN
98,21.4,Water and Sanitation,Domestic Water Consumption,Total domestic water consumption per capita (l...,Core,164.5041,2014.0,L/d/capita,147.2080,2014.0,NaN
60,21.5,Water and Sanitation,Total Water Consumption,Total water consumption per capita (litres/day),Supporting,226.6007,2014.0,L/d/capita,502.0710,2014.0,NaN
99,21.6,Water and Sanitation,Water Service Interruptions,Average annual hours of water service interrup...,Supporting,NaN,NaN,hrs/yr/household,0.0929,2015.0,NaN
28,21.7,Water and Sanitation,Water Loss,Percentage of water loss (unaccounted for water),Supporting,10.6329,2014.0,%,10.6329,2014.0,NaN


- Both Greater Melbourne and the Melbourne municipality achieved 100% coverage of potable water service in 2014. This indicates that all residents have access to safe and drinkable water, which is essential for public health and well-being.
- Similarly, 100% of the city population had improved water access in 2014, ensuring sustainable and reliable access to clean water sources. This reflects efforts to provide adequate infrastructure and services for water supply across both areas.
- In 2014, both areas reported 100% access to improved sanitation facilities, such as toilets and sewage systems. This indicates widespread access to hygienic sanitation facilities, contributing to public health and environmental sanitation.
- Greater Melbourne had a domestic water consumption rate of 164.5041 liters per day per capita in 2014, while the Melbourne municipality reported a slightly lower rate of 147.2080 liters per day per capita. Variations in consumption rates may reflect differences in household sizes, lifestyles, and water use behaviors between urban and suburban areas.
- The total water consumption per capita was notably higher in Greater Melbourne (226.6007 liters per day per capita) compared to the Melbourne municipality (502.0710 liters per day per capita) in 2014. This disparity may be attributed to factors such as industrial water use, commercial activities, and irrigation practices, which could be more prevalent in the urbanized region of Greater Melbourne.
- Both Greater Melbourne and the Melbourne municipality reported a water loss percentage of 10.6329% in 2014. Water loss, or unaccounted-for water, can occur due to leaks, theft, or inefficient distribution systems. Efforts to minimize water loss are crucial for conserving water resources and ensuring sustainable water management practices.

# Part 2: Conclusions

Based on the provided information, here are recommendations for the city council to address various aspects within the Melbourne municipality:

- Employment Initiatives and Poverty Alleviation: Given the disparity in employment opportunities and poverty rates compared to Greater Melbourne, the council should prioritize targeted employment initiatives and poverty alleviation measures. This could involve partnering with local businesses to create job opportunities, providing vocational training programs, and establishing support systems for unemployed individuals.
- Education Support Programs: To address differences in education completion rates, especially at the secondary level, the council should implement support programs for students within the municipality. This may include tutoring services, mentorship programs, and initiatives to improve teacher allocations and student-teacher ratios.
- Infrastructure and Service Reliability: Prioritize infrastructure improvements, particularly in areas like electrical services where disparities exist between the Melbourne municipality and Greater Melbourne. This could involve targeted investments in infrastructure maintenance and upgrades to enhance service reliability.
- Healthcare Access and Equity: Invest in expanding healthcare infrastructure, attracting healthcare professionals, and implementing strategies to improve healthcare outcomes, particularly in areas with lower healthcare staffing levels and higher rates of homelessness.
- Public Safety and Crime Prevention: Increase police presence, enhance community policing efforts, and implement crime prevention initiatives tailored to the specific needs of each area to address disparities in crime rates and public safety concerns.
- Homelessness Prevention and Support: Focus on expanding affordable housing programs, increasing funding for supportive services, and implementing homelessness prevention strategies to mitigate disparities in homelessness rates between the Melbourne municipality and Greater Melbourne.
- Transportation Accessibility and Sustainability: Invest in expanding public transit infrastructure, promoting alternative transportation modes, enhancing road safety measures, and improving accessibility to transportation options, particularly in Greater Melbourne where coverage is comparatively lower.
- Environmental Sustainability: Continue efforts to monitor and improve air quality standards, promote sustainable transportation options, and implement measures to preserve and enhance green spaces and urban tree canopy.
- Water and Sanitation Management: Ensure comprehensive infrastructure for wastewater collection and treatment, potable water service, and sanitation facilities to maintain public health and environmental sustainability standards across the municipality.
- Data-driven Decision Making and Collaboration: Emphasize the importance of data-driven decision-making processes and collaboration with relevant stakeholders, including community groups, businesses, healthcare providers, and educational institutions, to effectively address disparities and promote equitable development across the Melbourne municipality.